In [16]:
"""
Author: Anmolbir Mann 
Email: amann33@gatech.edu 
CS7641 Assignment 3: Unsupervised Learning

This Notebook is mainly for exploring/developing 
see/use final python script, main.py to generate final results
"""

%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import json
import time

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display
plt.ion()

from sklearn.metrics import silhouette_score, mean_squared_error
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA, FastICA, FactorAnalysis
from sklearn.random_projection import GaussianRandomProjection
from unsupervised_learner import process_data, searchk_kmeans, search_components_gm, run_pca, plot_data, plot_tsne, run_ica
from sklearn.manifold import TSNE
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

RANDOM_SEED = 1994540101
np.random.seed(RANDOM_SEED) # keep results consistent

In [17]:
data = pd.read_csv("data/diabetes.csv")
x = data.iloc[:,:-1]
y = data.iloc[:,-1]

x_train, x_test, y_train, y_test = process_data(x, y)

In [ ]:
inertias, clusters_range, best_n_clusters = searchk_kmeans(x_train)

In [ ]:
plot_config = {
    "x": clusters_range,
    "y": inertias,
    "xlabel": 'Number of clusters',
    "ylabel": 'Sum of Squared Distances',
    "title": 'K Means: Number of Clusters vs Sum of Squared Distances'
}
plot_data(**plot_config)

In [ ]:
result = KMeans(n_clusters=best_n_clusters, init='random', n_init=10, max_iter=100, tol=1e-04, random_state=RANDOM_SEED)
km = result.fit(x_train)

In [ ]:
result.n_iter_ < result.max_iter # convergence

In [ ]:
plot_tsne(x_train, result.labels_, best_n_clusters, title=f"TSNE Visualization K means labeling with {best_n_clusters} clusters")